## Requirements
Configures the libraries for Amazon Bedrock, Amazon OpenSearch Serverless, and visualization libraries

In [ ]:
!pip install boto3>=1.28.57
!pip install awscli>=1.29.57
!pip install botocore>=1.31.57
!pip install opensearch-py==2.3.1
!pip install pypdf>=3.8,<4
!pip install matplotlib==3.8.2

In [ ]:
!python3 -m pip install boto3
!python3 -m pip install awscli
!python3 -m pip install botocore


## Create a CloudFront for the image_url 
Create a cloudfront URL for the images to be accessible


In [ ]:

import os
import boto3
os.environ['AWS_PROFILE'] = 'aws-us-west-2'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2' 
print(os.getenv('AWS_PROFILE'))

session = boto3.Session(profile_name='aws-us-west-2')
# Create an STS client
sts = session.client('sts')

# Get the current account ID
account_id = sts.get_caller_identity()['Account']

# Print the account ID
print(f"AWS Account ID: {account_id}")



In [ ]:
# Create a bucket for the images to be store
import os
import boto3
from botocore.exceptions import ClientError

# Construct the bucket name using the AWS Account ID and a string
bucketname = f"{account_id}-vectorimagestore"

# Create an S3 resource
s3 = session.resource('s3')

# Create the S3 bucket
try:
    bucket = s3.create_bucket(
        Bucket=bucketname,
        CreateBucketConfiguration={
            'LocationConstraint': session.region_name
        }
    )
    print(f"Created S3 bucket: {bucketname}")
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f"Bucket {bucketname} already exists.")
    else:
        raise e

# Create a CloudFront client
cf = session.client('cloudfront')

try:
    # Create the origin access identity
    response = cf.create_cloud_front_origin_access_identity(
        CloudFrontOriginAccessIdentityConfig={
            'CallerReference': str(hash(bucketname)),
            'Comment': 'Access identity for ' + bucketname
        }
    )

    origin_access_identity_id = response['CloudFrontOriginAccessIdentity']['Id']
    origin_access_identity_arn = response['CloudFrontOriginAccessIdentity']['S3CanonicalUserId']

    # Configure the S3 origin
    origin = {
        'Id': bucketname,
        'DomainName': f'{bucketname}.s3.amazonaws.com',
        'S3OriginConfig': {
            'OriginAccessIdentity': f'origin-access-identity/cloudfront/{origin_access_identity_id}'
        }
    }

    # Create the distribution configuration
    distribution_config = {
        'CallerReference': str(hash(bucketname)),
        'Comment': 'Distribution for ' + bucketname,
        'Origins': {
            'Quantity': 1,
            'Items': [origin]
        },
        'DefaultCacheBehavior': {
            'TargetOriginId': bucketname,
            'ViewerProtocolPolicy': 'redirect-to-https',
            'AllowedMethods': {
                'Quantity': 2,
                'Items': ['GET', 'HEAD'],
                'CachedMethods': {
                    'Quantity': 2,
                    'Items': ['GET', 'HEAD']
                }
            },
            'Compress': True,
            'DefaultTTL': 86400,
            'MaxTTL': 31536000,
            'MinTTL': 0,
            'ForwardedValues': {
                'QueryString': False,
                'Cookies': {
                    'Forward': 'none'
                },
                'Headers': {
                    'Quantity': 0
                },
                'QueryStringCacheKeys': {
                    'Quantity': 0
                }
            }
        },
        'Enabled': True
    }

    # Create the CloudFront distribution
    response = cf.create_distribution(
        DistributionConfig=distribution_config
    )

    print(response)

    # Print the distribution URL
    distribution_url = response['Distribution']['DomainName']
    print(f"CloudFront distribution URL: https://{distribution_url}")
except ClientError as e:
    if e.response['Error']['Code'] == 'DistributionAlreadyExists':
        print("CloudFront distribution already exists.")
    else:
        raise e

## Update the image_catalog URL base path
Update the json with the new CloudFront Distribution base URL stored in ```distribution_url```

In [ ]:
import os
import json

# Set the path to the image catalog file
image_catalog_file = "image_data/image_catalog.json"

# Read the existing JSON data from the file
with open(image_catalog_file, "r") as file:
    data = json.load(file)

# Update the "image_url" field with the CloudFront distribution URL
for item in data:
    item["image_url"] = item["image_url"].replace("cloudfrontURL", distribution_url)

# Write the updated data back to the file
with open(image_catalog_file, "w") as file:
    json.dump(data, file, indent=4)

print("Image catalog file updated successfully.")

## Upload image_data to S3 and push to CloudFront distribition
Copy the data from ./image_data to S3 bucket
Invalidate the distribution cache sending it to CloudFront

In [ ]:
import os
import boto3
from botocore.exceptions import ClientError

# AWS credentials and region
AWS_PROFILE = 'aws-us-west-2'
AWS_REGION = 'us-west-2'

# S3 bucket name
bucketname = f"{account_id}-vectorimagestore"

# CloudFront distribution ID
CLOUDFRONT_DISTRIBUTION_ID = 'E2J2I9LLRVP1CK'

# Create S3 and CloudFront clients
session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')
cloudfront = session.client('cloudfront')

# Upload files to S3
def upload_to_s3(local_dir, s3_prefix):
    for root, dirs, files in os.walk(local_dir):
        for file in files:
            local_path = os.path.join(root, file)
            s3_path = os.path.join(s3_prefix, os.path.relpath(local_path, local_dir))
            try:
                s3.upload_file(local_path, bucketname, s3_path)
                print(f"Uploaded {local_path} to s3://{bucketname}/{s3_path}")
            except ClientError as e:
                print(f"Error uploading {local_path}: {e}")

# Invalidate CloudFront distribution cache
def invalidate_cloudfront_cache(paths):
    try:
        response = cloudfront.create_invalidation(
            DistributionId=CLOUDFRONT_DISTRIBUTION_ID,
            InvalidationBatch={
                'Paths': {
                    'Quantity': len(paths),
                    'Items': paths
                },
                'CallerReference': 'unique-reference-id'
            }
        )
        print(f"Invalidated CloudFront distribution cache for {', '.join(paths)}")
    except ClientError as e:
        print(f"Error invalidating CloudFront distribution cache: {e}")

# Upload image data to S3
upload_to_s3('./image_data', '')

# Invalidate CloudFront distribution cache
invalidate_cloudfront_cache(['/*'])


## Update Bucket Policy for CloudFront Distribution
The policy provides access for CloudFront to the objects stored in the bucket.

**NOTE** NOT COMPLETE

In [ ]:
# Create a bucket policy to allow CloudFront access
print(f"Creating bucket policy for CloudFront Origin Access Identity: {CLOUDFRONT_DISTRIBUTION_ID}")

# Get the CloudFront Origin Access Identity (OAI)
try:
    response = cloudfront.get_cloud_front_origin_access_identity_config(
        Id=CLOUDFRONT_DISTRIBUTION_ID
    )
    oai_arn = response['CloudFrontOriginAccessIdentityConfig']['S3CanonicalUserId']
except ClientError as e:
    print(f"Error getting CloudFront OAI: {e}")
    exit(1)

print(f"CloudFront Response: {response}")

bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::cloudfront:user/CloudFront Origin Access Identity {CLOUDFRONT_DISTRIBUTION_ID}"
            },
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{bucketname}/*"
        }
    ]
}

try:
    s3.put_bucket_policy(Bucket=bucketname, Policy=json.dumps(bucket_policy))
    print(f"Bucket policy created for {bucketname}")
except ClientError as e:
    print(f"Error creating bucket policy: {e}")

In [ ]:
{
    "Version": "2008-10-17",
    "Id": "PolicyForCloudFrontPrivateContent",
    "Statement": [
        {
            "Sid": "1",
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::cloudfront:user/CloudFront Origin Access Identity E3BVAIXSO0D205"
            },
            "Action": "s3:GetObject",
            "Resource": "arn:aws:s3:::682723651788-vectorimagestore/*"
        }
    ]
}

## Generate embeddings for image data and store embeddings in local JSON files
1. Initialize the boto3 client for Amazon Bedrock to use the Bedrock API's for invoking the Amazon Titan Multimodal Embeddings model
2. Generate embeddings for the images and their corresponding product titles in a shared embedding space by invoking the Amazon Titan Multimodal Embeddings Model on the tourism dataset represented in ./image_data/image_catalog.json files
3. Save the generated embeddings in flat JSON files

**NOTE:**
(code removed) After the above code snippet is executed, here is the structure of the embedding files generated in my local directory. These embeddings represent the vector representations of the Indo-fashion images and their product titles in a shared embedding space. Each vector representation generated by the Amazon Titan Multimodal Embeddings model has 1024 dimensions which is the default value, but other dimensions are available

 ```
 
 [
  {
    "image_path": "001.jpg",
    "image_title": "Driver Career",
    "image_labels": "Bus Driver Transport Green",
    "image_class": "tower",
    "image_url": "https://d9yx5bzoplulh.cloudfront.net/001.jpg",
    "multimodal_vector": [
      0.043503903,
      -0.011784887,
      -0.021839952,
      -0.014455186,
      ........
      ........
    ]
  },
  {
    "image_path": "002.jpg",
    "image_title": "Beach",
    "image_labels": "Beach Ocean Sand Swimming",
    "image_class": "sealink",
    "image_url": "https://d9yx5bzoplulh.cloudfront.net/002.jpg",
    "multimodal_vector": [
      0.014641403,
      0.017837116,
      0.0024839255,
      -0.013209195,
      0.021653477,
      0.015551946,
      ........
      ........
    ]
  }
]
```

## Create the OpenSearch serverless collection

Amazon OpenSearch Serverless has a new feature called Collections that I use below for storing the vector embeddings generated above. I initialized an Amazon OpenSearch Serverless Collection named “image-search-multimodal” as shown below. Additionally, I create the encryption policies, network policies, and data access policies for accessing the Amazon OpenSearch Serverless Collection. At the end of the code block below, I instantiate an Amazon OpenSearch Serverless client to connect to Amazon OpenSearch Serverless.

In [ ]:
# Create the opensearch serverless collection
# create the opensearch client

import boto3
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

os.environ['AWS_PROFILE'] = 'aws-us-west-2'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2' 
print(os.getenv('AWS_PROFILE'))

session = boto3.Session(profile_name='aws-us-west-2')

vector_store_name = 'image-search-multimodal'
index_name = "image-search-multimodal-index"
encryption_policy_name = "image-search-multimodal-ep"
network_policy_name = "image-search-multimodal-np"
access_policy_name = 'image-search-multimodal-ap'
identity = session.client('sts').get_caller_identity()['Arn']

#aoss_client = boto3.client('opensearchserverless')
aoss_client = session.client('opensearchserverless')

security_policy = aoss_client.create_security_policy(
    name = encryption_policy_name,
    policy = json.dumps(
        {
            'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AWSOwnedKey': True
        }),
    type = 'encryption'
)

network_policy = aoss_client.create_security_policy(
    name = network_policy_name,
    policy = json.dumps(
        [
            {'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AllowFromPublic': True}
        ]),
    type = 'network'
)

collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

while True:
    status = aoss_client.list_collections(collectionFilters={'name':vector_store_name})['collectionSummaries'][0]['status']
    if status in ('ACTIVE', 'FAILED'): break
    time.sleep(10)

access_policy = aoss_client.create_access_policy(
    name = access_policy_name,
    policy = json.dumps(
        [
            {
                'Rules': [
                    {
                        'Resource': ['collection/' + vector_store_name],
                        'Permission': [
                            'aoss:CreateCollectionItems',
                            'aoss:DeleteCollectionItems',
                            'aoss:UpdateCollectionItems',
                            'aoss:DescribeCollectionItems'],
                        'ResourceType': 'collection'
                    },
                    {
                        'Resource': ['index/' + vector_store_name + '/*'],
                        'Permission': [
                            'aoss:CreateIndex',
                            'aoss:DeleteIndex',
                            'aoss:UpdateIndex',
                            'aoss:DescribeIndex',
                            'aoss:ReadDocument',
                            'aoss:WriteDocument'],
                        'ResourceType': 'index'
                    }],
                'Principal': [identity],
                'Description': 'Easy data policy'}
        ]),
    type = 'data'
)

host = collection['createCollectionDetail']['id'] + '.' + os.environ.get("AWS_DEFAULT_REGION", None) + '.aoss.amazonaws.com:443'
print(host)
service = 'aoss'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, os.environ.get("AWS_DEFAULT_REGION", None), service)
host_parts = host.split(':')
client = OpenSearch(
    hosts = [{'host': host_parts[0], 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)


## Create the vector index
Amazon OpenSearch is a versatile and fully managed suite for search and analytics, that offers robust scalability. It supports KNN (K-Nearest Neighbors) search, enabling retrieval of similar documents based on vectors. To use this capability, I create a new vector index within the vector search collection. Each document within this index will encompass six key properties: "image_path," "image_title," "image_labels," "image_class," "image_url," and a vector embeddings field named "multimodal_vector". The dense vector embeddings, generated from the Indo Fashion dataset by the Amazon Titan Multimodal Embeddings model, will contain 1024 dimensions. To query these vectors, the index "image-search-multimodal-index" is configured to use the Non-Metric Space Library (nmslib), with the Hierarchical Navigable Small Worlds algorithm (HNSW) and cosine similarity (cosinesimil)

In [ ]:
#create the vector index

import json
index_name = "image-search-multimodal-index" 

index_body = {
    "mappings": { 
        "properties": {
            "image_path": {"type": "text"}, 
            "image_title": {"type": "text"}, 
            "image_labels": {"type": "text"}, 
            "image_class": {"type": "text"}, 
            "image_url": {"type": "text"}, 
            "multimodal_vector": {
                "type": "knn_vector", 
                "dimension": 1024, 
                "method": 
                {
                    "engine": "nmslib",
                    "space_type": "cosinesimil",
                    "name": "hnsw",
                    "parameters": {"ef_construction": 512, "m": 16},
                }, 
            },
        }
    },
    "settings": {
        "index": {
            "number_of_shards": 2, 
            "knn.algo_param": {"ef_search": 512}, 
            "knn": True,
        } 
    },
}

try:
    response = client.indices.create(index_name, body=index_body) 
    print(json.dumps(response, indent=2))
except Exception as ex: print(ex)

# describe new vector index
try:
    response = client.indices.get("image-search-multimodal-index") 
    print(json.dumps(response, indent=2))
except Exception as ex: print(ex)

**NOTE**
You can also review the new Vectorsearch collection, "image-search-multimodal", and the associated vector index, "image-search-multimodal-index", from the ([Amazon OpenSearch Service console](https://console.aws.amazon.com/aos/home/)).

## Read embeddings from flat files and insert them into Amazon OpenSearch
In the next code snippet below, I ingest the JSON Documents generated in Step 2 into the Amazon OpenSearch index "image-search-multimodal-index".

In [ ]:
# Read embeddings from flat file and insert into opensearch

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from opensearchpy import helpers
from opensearchpy.helpers import bulk

# Directory containing the JSON files
json_files_directory = "./"

# Iterate through each JSON file
for filename in os.listdir(json_files_directory):
    if filename.startswith("embedding_requests") and filename.endswith(".json"):
        file_path = os.path.join(json_files_directory, filename)

        # Load JSON file
        with open(file_path, "r") as file:
            data = json.load(file)

        print(f"Starting indexing for :: {filename}")

        # Use the bulk API to insert documents for the current file
        success, failed = bulk(
            client,
            data,
            index="image-search-multimodal-index",  # Replace with your OpenSearch index name
            raise_on_exception=True
        )

        print(f"Indexed {success} documents successfully, {failed} documents failed for file: {filename}")

Once the documents are ingested into the index, the Amazon OpenSearch Collections index **product-search-mulitmodal-index** looks like:

**Vector Fields**
* Vector Field Name: multimodal_vector
* Engine: nmslib
* Dimensions: 1024
* Distance Type: cosine
* M: 16
* ef_construction: 512
* ef_search: 512


**Metadata**
* image_labels
* image_class
* image_path
* image_title
* image_url
